In [4]:
# blab init
import blab
startup_notebook = blab.blab_startup()
%run $startup_notebook

blab init
blab found local libs at D:\Dropbox\31_Projekte\01_Python\libs
Start Time: 22:04:18


time: 277 ms


## Rank Rows
* `rank`: Select the max row per group. Or the min.<br>
   Or mark the rows instead of selecting them. 

In [5]:
import numpy      as np
import pandas     as pd 
import bpyth      as bpy
import pandasklar as pak 

import time
from pandas._testing import assert_frame_equal

time: 318 ms


In [6]:
# Create test data
# cols_group = ['AA']     # Very few very large groups
# cols_group = ['A']      # Few large groups
# cols_group = ['B']      # Many small groups

np.random.seed(10)
anz = 100000 #0
df = pak.zufallsdaten(anz)
df['AA'] = df.A.apply(lambda x: int(x/10)*10 )
df['AA'] = pak.change_datatype(df['AA'] )
df = pak.move_cols(df,'AA')
df = pak.rename_col(df,'C','other_data')
df = pak.rename_col(df,'D','score')
df
#pak.analyse_cols(df)

,AA,A,B,other_data,score
0,40,40,97438,0.369,9.97
1,30,33,87940,0.186,5.87
2,30,32,95268,0.063,3.27
3,20,27,61350,0.551,5.84
4,30,33,32778,0.393,1.92
...,...,...,...,...,...
99995,30,33,52307,0.320,8.20
99996,30,32,17225,0.366,6.87
99997,30,31,72053,0.583,4.13
99998,20,29,97320,0.443,7.72


time: 1.94 s


## rank()

In [7]:
?pak.rank

time: 266 ms


Signature:
pak.rank(
    df,
    cols_group,
    col_score,
    find='max',
    col_target='',
    on_conflict='first',
)
Docstring:
Select the max row per group. Or the min.
Or mark the rows instead of selecting them. 
* cols_group:   Name of the columns to be grouped by.
* col_score:    Name of the column whose minimum or maximum is to be found.
* find:         'min' or 'max', default is 'max'.
* col_target:   Should a ranking column be added? If yes, then give the name here.
                If col_target is given, then the dataframe is returned completely, plus the new column.
                If no col_target is given, only rank 1 is returned.
* on_conflict:  How to rank the group of records that have the same value. 
                Possible values: 'min','max','average','dense' and 'first', see pandas rank.
File:      d:\dropbox\31_projekte\01_python\git\pandasklar\src\pandasklar\rank.py
Type:      function


In [8]:
# select the rows with max score, grouped by AA
ranked = pak.rank( df, cols_group=['AA'], col_score='score')
ranked

,AA,A,B,other_data,score
567,20,26,54706,0.479,9.99
2072,30,36,48311,0.574,9.99
2298,40,40,55534,0.408,9.99


time: 261 ms


In [9]:
# select the rows with min score, grouped by AA
ranked = pak.rank( df, cols_group=['AA'], col_score='score', find='min')
ranked

,AA,A,B,other_data,score
886,30,32,12017,0.863,0.0
1058,20,25,63125,0.128,0.0
50593,40,40,78253,0.662,0.0


time: 268 ms


### mark instead of select

In [10]:
# Instead of selecting the top rows, we can mark all rows by it's rank.
# The one with max score in the group gets rank 1.
# This time we group by B.

df = pak.rank( df, cols_group=['B'], col_score='score', col_target='rank')
df.sort_values(['B','score'], ascending=[True,False])

,AA,A,B,other_data,score,rank
47467,30,36,10000,0.465,4.42,1
1678,30,31,10001,0.595,8.25,1
66891,20,25,10004,0.555,7.38,1
15032,30,36,10005,0.591,8.97,1
87010,20,25,10006,0.281,4.08,1
...,...,...,...,...,...,...
75134,30,32,99994,0.469,0.11,2
44375,20,26,99995,0.577,0.29,1
84106,30,36,99995,0.679,0.20,2
27591,20,27,99996,0.102,7.13,1


time: 370 ms


### Deal with ambiguous results 

In [11]:
# In this dataset there are many records with max score.
mask = (df.score >= 9.99)  &  (df.AA==20)
df[mask].head(5)

,AA,A,B,other_data,score,rank
567,20,26,54706,0.479,9.99,1
3865,20,20,21622,0.781,9.99,1
5834,20,28,24588,0.672,9.99,1
8084,20,24,77773,0.778,9.99,1
11781,20,29,10303,0.086,9.99,1


time: 243 ms


In [12]:
# on_conflict='first' is default.
df = pak.rank( df, cols_group=['AA'], col_score='score', col_target='rank')
df.sort_values(['AA','score'], ascending=[True,False])

,AA,A,B,other_data,score,rank
567,20,26,54706,0.479,9.99,1
3865,20,20,21622,0.781,9.99,2
5834,20,28,24588,0.672,9.99,3
8084,20,24,77773,0.778,9.99,4
11781,20,29,10303,0.086,9.99,5
...,...,...,...,...,...,...
92661,40,41,87336,0.393,0.03,4075
97927,40,41,76769,0.471,0.03,4076
74961,40,42,34412,0.743,0.02,4077
44135,40,40,92031,0.237,0.01,4078


time: 288 ms


In [13]:
# If you sort the dataframe before, you will get secondary ranking criterias.
# In this example the ranking criterias are score and B.
df = pak.rank( df.sort_values('B'), cols_group=['AA'], col_score='score', col_target='rank')
df.sort_values(['AA','score'], ascending=[True,False])

,AA,A,B,other_data,score,rank
11781,20,29,10303,0.086,9.99,1
70760,20,27,11885,0.398,9.99,2
89646,20,22,15461,0.293,9.99,3
26245,20,23,17271,0.158,9.99,4
95488,20,27,20148,0.373,9.99,5
...,...,...,...,...,...,...
97927,40,41,76769,0.471,0.03,4075
92661,40,41,87336,0.393,0.03,4076
74961,40,42,34412,0.743,0.02,4077
44135,40,40,92031,0.237,0.01,4078


time: 338 ms


In [14]:
# group by A, on_conflict='average' 
df = pak.rank( df, cols_group=['A'], col_score='score', col_target='rank', on_conflict='average')
a = df.sort_values(['A','score'], ascending=[True,False])
a

,AA,A,B,other_data,score,rank
3865,20,20,21622,0.781,9.99,1.0
77384,20,20,52557,0.081,9.98,2.5
566,20,20,94494,0.075,9.98,2.5
44876,20,20,76498,0.624,9.97,4.0
86925,20,20,18985,0.791,9.96,6.0
...,...,...,...,...,...,...
56713,40,42,25721,0.053,0.11,681.5
44926,40,42,31006,0.253,0.11,681.5
46278,40,42,26719,0.590,0.06,683.0
68928,40,42,14264,0.344,0.04,684.0


time: 305 ms


In [15]:
# group by A, on_conflict='min' 
df = pak.rank( df, cols_group=['A'], col_score='score', col_target='rank', on_conflict='min')
a = df.sort_values(['A','score'], ascending=[True,False])
a

,AA,A,B,other_data,score,rank
3865,20,20,21622,0.781,9.99,1
77384,20,20,52557,0.081,9.98,2
566,20,20,94494,0.075,9.98,2
44876,20,20,76498,0.624,9.97,4
86925,20,20,18985,0.791,9.96,5
...,...,...,...,...,...,...
56713,40,42,25721,0.053,0.11,681
44926,40,42,31006,0.253,0.11,681
46278,40,42,26719,0.590,0.06,683
68928,40,42,14264,0.344,0.04,684


time: 299 ms
